In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import pandas as pd
import requests
from io import StringIO
import re
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA 
import umap.umap_ as umap
import matplotlib.pyplot as plt
import hdbscan
pd.set_option('display.max_colwidth', None)

In [2]:
sheet_name_lsit = ['Sci/2', 'Sci_Extra/1', 'Sci/1', 'Sci_Extra/2', 'Engineering/1', 'Engineering_Extra/1', 'Engineering/2', 'Engineering_Extra/2', 'SIIT/2', 'SIIT/1', 'Innovation_Extra/1', 'Innovation_Extra/2', 'CIS_Extra/1', 'CIS/1', 'CIS_Extra/2', 'CIS/2']

In [3]:
sheet_id = "1ExtEUwVLi5Y_mmqxNd0oX7WrWzj8n6TeBLcQg7Xu0GU"
dataframe = pd.DataFrame(columns=["code", "desc", "add1", "add2", "add3","ma"])

for sheet_name in sheet_name_lsit:
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    response = requests.get(url)
    decoded_content = response.content.decode('utf-8')
    data_tem =  pd.read_csv(StringIO(decoded_content))
    dataframe = pd.concat([dataframe, data_tem], ignore_index=True)

print(dataframe.shape)
for i in dataframe.columns:
    dataframe[f"{i}"] = dataframe[f"{i}"].str.strip().str.lower()

print(dataframe.shape)

(4272, 6)
(4272, 6)


In [5]:
dataframe.head(20)

,code,desc,add1,add2,add3,ma
0,am100,"basics concepts in applied mathematics, mathematical modeling, operations research and computational mathematics",[],[],[],sci
1,am100,"basics concepts in applied mathematics, mathematical modeling, operations research and computational mathematics",[],[],[],sci
2,am100,concepts in applied mathematics and career paths. (study evaluation by s or u),[],[],[],sci
3,am101,"limit and continuity of single variable functions, the derivative and its applications, antiderivatives and indefinite integrals, techniques of integrations, definite integrals and applications of the definite integrals in geometry and physics, improper integrals and convergence tests, infinite series and convergence tests, power series and theorem of taylor series, software programme for calculus of single variable functions.",['disciplinary course(s): ไม่นับหน่วยกิตให้ผืที่กำลังศึกษาหรือสอบได้ ค.111 หรือ ค.211 หรือ ค.216 หรือ ค.218'],"['consecutive course(s): am102, st326']",[],sci
4,am101,"limits and continuity of single variable functions, derivatives of single variable functions and applications, antiderivatives and indefinite integrals, techniques of integrations, definite integrals and applications, improper integrals and convergence tests, infinite series and convergence tests, power series and taylor series, software for calculus of single variable functions. note: there is no credit for students who are currently taking or have earned credits of ma111 or ma113 or ma211 or ma216 or ma218",['disciplinary course(s): หมายเหตุ : ไม่นับหน่วยกิตให้ผู้ที่กำลังศึกษาหรือสอบได้ ค.111 หรือ ค.113 หรือ ค.211 หรือ ค.216 หรือ ค.218'],"['consecutive course(s): am102, am201, st326, st329']",[],sci
5,am102,"transformation of coordinate systems, polar coordinates and graphing, vectors, lines and planes in 3d- space, functions of several variables and suface plot, limits and continuity of several variables functions, partial derivatives, higher-order partial derivatives, implicit functions theorem, applications of partial derivatives in optimization, multiple integrations, differential vector calculus, introduction to line integrals and surface integrals, gauss’s theorem, green’s theorem and stokes’ theorem, software programme for calculus of several variable functions.",['disciplinary course(s): am101'],"['consecutive course(s): am201, am311, am327, ma316, ma351, ma412, st226, st321, st327']",[],sci
6,am102,"polar coordinates, vectors, lines, planes and surfaces in three dimensional space, limits and continuity of several variables functions, partial derivatives, higher-order partial derivatives, the chain rule, implicit differentiation, applications of partial derivatives, maxima and minima of several variables functions, multiple integrations, differential vector calculus, introduction to line integrals and surface integrals, gauss’s theorem, green’s theorem and stokes’ theorem, software for calculus of several variables functions.",['disciplinary course(s): am101\nสอบได้ คป.101 หมายเหตุ : ไม่นับหน่วยกิตให้ผู้ที่กำลังศึกษาหรือสอบได้ ค.112 หรือ ค.209 หรือ ค.213 หรือ ค.217 หรือ ค.219'],"['consecutive course(s): am231, am327, ma316, ma412, st226, st321, st327']",[],sci
7,am151,"symbolic logic, methods of proof, proof of quantifier sentences, real number system, properties of real numbers, sets, properties and theorems of sets, relations, equivalence relations, partial order relations, functions, and types of functions.","['consecutive course(s): am211, am457, ma332, ma337, ma338, ma366, ma368']",[],[],sci
8,am200,"logic, algebra of sets, relations and functions, recurrence relations, generating functions, basic concepts in graph theory, number representation in computer, boolean algebra and combinatorial circuits, finite-state machines, finite automata, basic concepts in algebraic system. note : 1. this subject is equivalent to cs 101, discrete structures. 2. there is no credit for students who currently taking or have earned credits o

In [6]:
dataframe.head()
dataframe.rename(columns={
    "desc":"description",
    "add1":"disciplinary course(s)",
    "add2":"consecutive course(s)",
    "ma":"faculty"
},inplace=True)
dataframe.drop(columns=["add3"],inplace=True)

In [7]:
dataframe["description"] = dataframe["description"].str.replace(r"[.,]", "", regex=True).str.strip()

dataframe.drop_duplicates(subset=["code"],inplace=True)
dataframe.dropna(subset=["description","code"],inplace=True)


In [8]:
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("คพ.","cs")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("คพ","cs")

dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("ทก.","at")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("ทก","at")


dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("คป.","am")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("คป","am")


dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("ทช.","bt")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("ทช","bt")


dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("วท.","sc")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("วท","sc")


dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("คม.","cm")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("คม","cm")


dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("ค.","ma")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("ค","ma")

dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("ส.","st")
dataframe["disciplinary course(s)"] = dataframe["disciplinary course(s)"].str.replace("ส","st")




In [9]:
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("คพ.","cs")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("คพ","cs")

dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("ทก.","at")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("ทก","at")


dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("คป.","am")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("คป","am")


dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("ทช.","bt")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("ทช","bt")


dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("วท.","sc")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("วท","sc")


dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("คม.","cm")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("คม","cm")


dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("ค.","ma")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("ค","ma")

dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("ส.","st")
dataframe["consecutive course(s)"] = dataframe["consecutive course(s)"].str.replace("ส","st")




In [14]:
def extract_courses(row):
    pattern = r"[A-Za-zก-ฮ]{1,2}\.? ?\d{3}"
    listt = []
    if isinstance(row, str):
        matches = re.findall(pattern, row)  
        listt.extend(matches)
        return listt
    return None 

dataframe["cancel_course"] = dataframe["disciplinary course(s)"].apply(extract_courses) + dataframe["consecutive course(s)"].apply(extract_courses)
dataframe.head()

,code,description,disciplinary course(s),consecutive course(s),faculty,cancel_course
0,am100,basics concepts in applied mathematics mathematical modeling operations research and computational mathematics,[],[],sci,[]
3,am101,limit and continuity of single variable functions the derivative and its applications antiderivatives and indefinite integrals techniques of integrations definite integrals and applications of the definite integrals in geometry and physics improper integrals and convergence tests infinite series and convergence tests power series and theorem of taylor series software programme for calculus of single variable functions,['disciplinary course(s): ไม่นับหน่วยกิตให้ผืที่กำลังศึกษาหรือstอบได้ ma111 หรือ ma211 หรือ ma216 หรือ ma218'],"['consecutive course(s): am102, st326']",sci,"[ma111, ma211, ma216, ma218, am102, st326]"
5,am102,transformation of coordinate systems polar coordinates and graphing vectors lines and planes in 3d- space functions of several variables and suface plot limits and continuity of several variables functions partial derivatives higher-order partial derivatives implicit functions theorem applications of partial derivatives in optimization multiple integrations differential vector calculus introduction to line integrals and surface integrals gauss’s theorem green’s theorem and stokes’ theorem software programme for calculus of several variable functions,['disciplinary course(s): am101'],"['consecutive course(s): am201, am311, am327, ma316, ma351, ma412, st226, st321, st327']",sci,"[am101, am201, am311, am327, ma316, ma351, ma412, st226, st321, st327]"
7,am151,symbolic logic methods of proof proof of quantifier sentences real number system properties of real numbers sets properties and theorems of sets relations equivalence relations partial order relations functions and types of functions,"['consecutive course(s): am211, am457, ma332, ma337, ma338, ma366, ma368']",[],sci,"[am211, am457, ma332, ma337, ma338, ma366, ma368]"
8,am200,logic algebra of sets relations and functions recurrence relations generating functions basic concepts in graph theory number representation in computer boolean algebra and combinatorial circuits finite-state machines finite automata basic concepts in algebraic system note : 1 this subject is equivalent to cs 101 discrete structures 2 there is no credit for students who currently taking or have earned credits of cs 101,['disciplinary course(s): 1.วิชานี้เทียบเท่า(equivalent)กับวิชา cs101 โmaรงstร้างแบบไม่ต่อเนื่อง 2.ไม่นับหน่วยกิตให้ผู้ที่กำลังศึกษา หรือstอบได้cs101'],"['consecutive course(s): am346, am411, am448, am451, ma338, ma366, ma367']",sci,"[cs101, cs101, am346, am411, am448, am451, ma338, ma366, ma367]"


In [9]:
dataframe.drop("add3",inplace=True,axis=1)

In [10]:
dataframe.to_csv("dataset.csv")